In [1]:
import os
import shutil
import glob
from pathlib import Path
import json

In [5]:
FOLDER_NAME = 'cortexA53_aarch64cpu_tflite28'
os.makedirs(os.path.join(os.getcwd(), FOLDER_NAME), exist_ok=True)

fusion_rule_json = json.load(open(os.path.join(os.getcwd(), "fusion_rule_test", "results", "detected_fusion_rule.json"), 'r'))
fusion_rule_target = os.path.join(os.getcwd(), FOLDER_NAME, "fusion_rules.json")

# 'ks' KeyError workaround
for key in fusion_rule_json.keys():
    if 'BF_' in key and key.endswith('_conv'):
        if fusion_rule_json[key]['obey']:
            fusion_rule_json[key]['force_no_obey_workaround'] = True
            fusion_rule_json[key]['obey'] = False
            print(f"Force disabling {key}")
        
with open(os.path.join(os.getcwd(), FOLDER_NAME, "fusion_rules.json"), 'w') as f:
    json.dump(fusion_rule_json, f, indent=4)

Force disabling BF_add_conv
Force disabling BF_concat_conv


In [3]:
kernel_type_naming_map = {
    "conv_bn_relu": "conv-bn-relu",
    "dwconv_bn_relu": "dwconv-bn-relu",
    "fc_block": "fc",
    "globalavgpool_block": "global-avgpool",
    "hswish_block": "hswish",
    "relu_block": "relu",
    "se_block": "se",
    "split_block": "split",
    "add_block": "add",
    "add_relu": "addrelu",
    "maxpool_block": "maxpool",
    "avgpool_block": "avgpool",
    "bn_block": "bn",
    "bn_relu": "bnrelu",
    "channel_shuffle": "channelshuffle",
    "concat_block": "concat"
}

file_list = glob.glob(os.path.join(os.getcwd(), "predictor_build", "results", "Predictor_*_finegrained4.pkl"))
assert len(file_list) == 16
target_stems = []

for file_name in file_list:
    file_stem = Path(file_name).stem
    target_stem = kernel_type_naming_map[file_stem.replace('Predictor_', '').replace('_finegrained4', '')]
    print(f"{file_stem} -> {target_stem}")
    
    shutil.copy(
        file_name,
        os.path.join(os.getcwd(), FOLDER_NAME, f"{target_stem}.pkl")
    )
    target_stems.append(target_stem)

Predictor_avgpool_block_finegrained4 -> avgpool
Predictor_concat_block_finegrained4 -> concat
Predictor_channel_shuffle_finegrained4 -> channelshuffle
Predictor_fc_block_finegrained4 -> fc
Predictor_add_block_finegrained4 -> add
Predictor_hswish_block_finegrained4 -> hswish
Predictor_conv_bn_relu_finegrained4 -> conv-bn-relu
Predictor_se_block_finegrained4 -> se
Predictor_bn_relu_finegrained4 -> bnrelu
Predictor_dwconv_bn_relu_finegrained4 -> dwconv-bn-relu
Predictor_bn_block_finegrained4 -> bn
Predictor_add_relu_finegrained4 -> addrelu
Predictor_globalavgpool_block_finegrained4 -> global-avgpool
Predictor_relu_block_finegrained4 -> relu
Predictor_maxpool_block_finegrained4 -> maxpool
Predictor_split_block_finegrained4 -> split


In [4]:
# Create meta file containing key-value storage
META_FILENAME = os.path.join(os.getcwd(), FOLDER_NAME, "metadata.yml")
FOLDER_PWD = os.path.join(os.getcwd(), FOLDER_NAME)

header = f'''name: {FOLDER_NAME}
version: 1.0
category: cpu
package_location: {FOLDER_PWD}
kernel_predictors:
'''

with open(META_FILENAME, "w") as f:
    f.write(header)
    for stem_name in target_stems:
        f.write(f"    - {stem_name}\n")